# UHK SMAP ovalek 01

In [1]:
# install packages
%pip install numpy pandas tensorflow tensorflow_datasets scikit-learn scipy matplotlib tensorflow_io

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import os
print(os.listdir("./data"))

['small', 'photos_sanitized.csv', 'parameters_products_color_sanitized.csv', 'tmp_dataset', 'parameters_list_sanitized.csv', 'parameters_products_pattern_sanitized.csv']


In [3]:
import tensorflow as tf
# check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-12-14 07:38:36.329340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 07:38:36.621683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 07:38:36.621702: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-14 07:38:37.489282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

Num GPUs Available:  0


2022-12-14 07:38:38.459608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 07:38:38.459870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 07:38:38.459920: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-14 07:38:38.459957: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-14 07:38:38.459992: W tensorflow/c

## Load data

In [4]:
import csv
import numpy as np
import os
import PIL
import PIL.Image

# load labels
label_ids = []
label_names = []
with open('./data/parameters_list_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in label_ids:
                label_ids += [row[0]]
                label_names += [row[2]]

# load product label relations
products_labels = {}
with open('./data/parameters_products_color_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in products_labels:
                products_labels[row[0]] = []
            for label_index, label_id in enumerate(label_ids):
                if label_id == row[2]:
                    products_labels[row[0]] += [label_names[label_index]]
with open('./data/parameters_products_pattern_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            line_count += 1
            if not row[0] in products_labels:
                products_labels[row[0]] = []
            for label_index, label_id in enumerate(label_ids):
                if label_id == row[2]:
                    products_labels[row[0]] += [label_names[label_index]]

# load products with main photo
products = []
with open('./data/photos_sanitized.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            line_count += 1
            if row[4] != 'Y':
                continue
            found = False
            for product in products:
                if product[0] == row[0]:
                    found = True
                    break
            if not found:
                products += [(row[0], row[1], row[3], row[4], ), ]
    print(f'Processed {line_count} lines.')
    print(f'Found products {products.__len__}.')


Column names are id_product, id_photo, date_update, position, show_in_lead
Processed 16161 lines.
Found products <method-wrapper '__len__' of list object at 0x7f21fa56a100>.


In [5]:
import os
from os.path import exists
from PIL import Image
import imghdr

# prepare data for dataset
data_photos_paths = []
data_labels = []
img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for product in products:
    tmp_path = f'./data/small/{int(product[0]) % 10}/{product[0]}/{product[0]}_{product[1]}_{product[3]}_{product[2]}_small.webp'
    tmp_path_jpg = f'./data/small/{int(product[0]) % 10}/{product[0]}/{product[0]}_{product[1]}_{product[3]}_{product[2]}_small.jpg'
    if not exists(tmp_path):
        continue
    if not exists(tmp_path_jpg) and os.stat(tmp_path).st_size == 0:
        print(f'Empty file: {tmp_path}')
        continue
    if not exists(tmp_path_jpg):
        im = Image.open(tmp_path).convert('RGB')
        im.save(tmp_path_jpg, "jpeg")

    img_type = imghdr.what(tmp_path_jpg)
    if img_type is None:
        print(f"{tmp_path_jpg} is not an image")
        continue

    data_photos_paths += [tmp_path_jpg]
    tmp_product_labels = products_labels[product[0]]
    tmp_labels_vector = []
    for label in label_names:
        tmp_labels_vector += [1 if label in tmp_product_labels else 0]
    data_labels += [tmp_labels_vector]
    # for label_index, label_name in enumerate(label_names):
    #     if label_name == tmp_product_labels[0]:
    #         data_labels += [label_index]
    #         break

# print(', '.join(data_labels[0]))

## Prepare train test and validation data

In [6]:
from sklearn.model_selection import train_test_split
import tensorflow_io as tfio

data_photos_paths_train, data_photos_paths_tmp, data_labels_train, data_labels_tmp = train_test_split(data_photos_paths, data_labels, test_size=0.25, random_state=42)
data_photos_paths_test, data_photos_paths_validation, data_labels_test, data_labels_validation = train_test_split(data_photos_paths_tmp, data_labels_tmp, test_size=0.4, random_state=None, shuffle=False)



def _create_dataset(X, y):
    # X = tf.constant(X)
    # y = tf.constant(y)
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    def _prepare_picture_func(picturepath, labels):
        image_string = tf.io.read_file(picturepath)
        image = tf.image.decode_image(image_string, channels=3)
        # print(picturepath)
        # image = tfio.image.decode_webp(image_string)
        # image = image[:,:,:3] # discard alpha channel
        image.set_shape([None, None, None])
        image = tf.image.resize(image, [250, 340])
        # image = tf.subtract(image, 116.779) # Zero-center by mean pixel
        # # image.set_shape([200, 200, 3])
        # image.set_shape((250, 250, 3))
        image.set_shape((250, 340, 3))
        # image = image.reshape(-1, 250, 340, 3)
        # image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
        # d = dict(zip([picturepath], [image])), labels
        # return (picturepath, image), labels
        return image, labels
        # return d
    return dataset.map(_prepare_picture_func)

train_ds = _create_dataset(data_photos_paths_train, data_labels_train)
test_ds = _create_dataset(data_photos_paths_test, data_labels_test)
validation_ds = _create_dataset(data_photos_paths_validation, data_labels_validation)


# data_photos_paths = tf.constant(data_photos_paths)
# data_labels = tf.constant(data_labels)
# dataset = tf.data.Dataset.from_tensor_slices((data_photos_paths, data_labels))

# def _prepare_picture_func(picturepath, labels):
#     image_string = tf.io.read_file(picturepath)
#     image = tf.image.decode_image(image_string, channels=3)
#     image.set_shape([None, None, None])
#     image = tf.image.resize(image, [150, 150])
#     image = tf.subtract(image, 116.779) # Zero-center by mean pixel
#     image.set_shape([150, 150, 3])
#     # image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
#     # d = dict(zip([picturepath], [image])), labels
#     return (picturepath, image), labels
#     # return d
# dataset = dataset.map(_prepare_picture_func)



# import tensorflow_datasets as tfds

# dataset = dataset.shuffle(buffer_size=100, seed=19575364)
# dataset.save('./data/tmp_dataset')



# train_set, test_set, valid_set = tfds.load('./data/tmp_dataset', 
#                                            split=["train[0%:75%]", "test[75%:90%]", "valid[90%:100%]"],
#                                            as_supervised=True, with_info=True, download=False)

#batch = dataset.batch(20)


# train_set, tmp_set = tf.keras.utils.split_dataset(
#     dataset, left_size=0.75, right_size=0.25, shuffle=True, seed=19575364
# )
# test_set, valid_set = tf.keras.utils.split_dataset(
#     dataset, left_size=0.60, right_size=0.40, shuffle=False, seed=None
# )


2022-12-14 07:38:42.677118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# train_batch = train_ds.batch(32)
# test_batch = test_ds.batch(32)

# # check batch
# for image_batch, labels_batch in train_batch:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break


## Create NN model

In [8]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


# # Initializes a sequential model
# model = Sequential()

# # First layer (pictures are 240x340=85000 x 3 colors => 255000)
# # model.add(Dense(2048, activation='relu', input_shape=(255000, )))
# model.add(Dense(2048, activation='relu', input_shape=(250, 250, 3)))
# # model.add(Dense(2048, activation='relu', input_shape=(187500, )))
# # model.add(Dense(2048, activation='relu', input_dim=187500))

# # Second layer
# model.add(Dense(256, activation='relu'))

# # Output layer (multi-label classification therefore use sigmoid activation function)
# model.add(Dense(48, activation='sigmoid'))

# model.summary()


num_classes = len(label_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(250, 340, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='sigmoid'),
  # layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
  # layers.Dense(1)
])

## Compile NN

In [9]:
# multi-label classification therefore use binary_crossentropy
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 250, 340, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 250, 340, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 170, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 170, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 85, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 85, 64)        1

## Use NN for data

In [10]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

# model.fit_generator(
#         train_batch, 
#         steps_per_epoch=len(train_batch), # batch_size
#         validation_data=test_batch,
#         validation_steps=len(test_batch), # batch_size
#         epochs=10,
#         callbacks=callbacks
#     )
# epochs=10
# history = model.fit(
#   train_ds,
#   validation_data=validation_ds,
#   epochs=epochs
# )
epochs=10
history = model.fit(
  train_ds.batch(32),
  validation_data=validation_ds.batch(32),
  epochs=epochs,
  callbacks=callbacks
)

Epoch 1/10


2022-12-14 07:38:43.653755: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2720000000 exceeds 10% of free system memory.
2022-12-14 07:38:44.018259: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2720000000 exceeds 10% of free system memory.
2022-12-14 07:38:44.380295: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2720000000 exceeds 10% of free system memory.
2022-12-14 07:38:49.790100: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2720000000 exceeds 10% of free system memory.
2022-12-14 07:38:50.548707: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2720000000 exceeds 10% of free system memory.


12/12 [==============================] - ETA: 0s - loss: 0.3988 - accuracy: 0.0168 
Epoch 1: val_loss improved from inf to 0.29144, saving model to model.h5
12/12 [==============================] - 126s 10s/step - loss: 0.3988 - accuracy: 0.0168 - val_loss: 0.2914 - val_accuracy: 0.0207 - lr: 0.0010
Epoch 2/10
12/12 [==============================] - ETA: 0s - loss: 0.2494 - accuracy: 0.0216 
Epoch 2: val_loss improved from 0.29144 to 0.20774, saving model to model.h5
12/12 [==============================] - 126s 11s/step - loss: 0.2494 - accuracy: 0.0216 - val_loss: 0.2077 - val_accuracy: 0.0207 - lr: 0.0010
Epoch 3/10
 1/12 [=>............................] - ETA: 1:56 - loss: 0.2082 - accuracy: 0.0300

KeyboardInterrupt: 